## Imports

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text  import TfidfVectorizer

## Loading and exploring dataset <br>
[Dataset link]('https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv')

In [ ]:
df = pd.read_csv('movies_metadata.csv', low_memory=False)
df.head()

In [ ]:
df.columns

In [ ]:
print(df['genres'][1])
print(type(df['genres'][1]))

In [ ]:
df[['original_title' , 'genres', 'tagline', 'overview']].isna().sum()

## Data cleaning

After I explored the dataset. I found many NaN and unwanted column - for our pupose,  in the dataset. So, I made subset of that are required. and will remove the Rows that  contain NaN<br>
`genres`, `tagline`, `overview`

In [ ]:
df_subset = df[['genres', 'tagline', 'overview']]
df_subset.head()

In [ ]:
df_subset = df_subset.reset_index(drop=True)
print(df_subset.shape)
df_subset.head()

Droping rows containing NaN values

In [ ]:
df_subset = df_subset.dropna()
df_subset.shape
df_subset

Converting the genre column to string

In [ ]:
def handel_genre(text):
    text = text.replace("'", '"')
    data = json.loads(text)
    return (' '.join(item['name'] for item in data )).lower()

.apply() function in pandas is used to apply a function in dataframe collumn  <br>
I have updated the 'genres' column with new one.

* "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]" <br>
TO <br>
* Adventure Fantasy Family

In [ ]:
df_subset['genres'] = df_subset['genres'].apply(handel_genre)
df_subset.head()

## Making the function to tokenize the texts and implementing in dataset.

- We have to tokenize the text while doing NLP because every words do not give us the context of sentence. <Br>
for instance, `he` `is` `a` boy`.` ONLY boy is our tag mean which gives us the general context. Same goes with punctuation. <br>
This will help in reasulting better accuray in most cases and reduce computations.

In [ ]:
def tokenize(text):
    stop_words = set(stopwords.words('english'))
    text_words = word_tokenize(text, 'english')
    text_token = [tokens for tokens in text_words if tokens.lower() not in stop_words and  tokens.lower() not in punctuation]
    return (' '.join(text_token)).lower()

In [ ]:
df_subset['tagline'] = df_subset['tagline'].apply(tokenize)
df_subset['overview'] = df_subset['overview'].apply(tokenize)
df_subset.head()

## Encoding genres For classification

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_encoded = pd.DataFrame(mlb.fit_transform(df_subset['genres'].apply(lambda x: x.split())), columns=mlb.classes_)
df_subset = pd.concat([df_subset, genre_encoded], axis=1)
df_subset = df_subset.drop('genres', axis=1)
df_subset.head()